Th goal of this notebook is to...

First we need to set up a number of variables we going to use later in the notebook:
- OPENAI_API_KEY: to access the OpenAI embeddings (secret)
- AZURESEARCH_ADMIN_KEY: to access the Azure Search service (secret)
- AZURESEARCH_ENDPOINT (theoretically it's not a secret, but it's better to keep it in .env file)
- AZURESEARCH_INDEX_NAME (theoretically it's not a secret, but it's better to keep it in .env file)
- if you create index in Azure Portal (not programatically), the default field names will be different than the ones assumed by the AzureSearch in langchain_community package. You can set the following variables to match the field names in your index:
    - AZURESEARCH_FIELDS_CONTENT_VECTOR=text_vector (it's not a secret) 
    - AZURESEARCH_FIELDS_ID=chunk_id (it's not a secret)
    - AZURESEARCH_FIELDS_CONTENT=chunk (it's not a secret)
    
All above will be read from .env file.

In [1]:
import os
from dotenv import load_dotenv, find_dotenv

The first thing we need is to build a retriever. 


We will use Microsoft AI Search with hybrid mode to retrieve the most relevant documents for a given query.

Source code for langchain_community.vectorstores.azuresearch: 
https://api.python.langchain.com/en/latest/_modules/langchain_community/vectorstores/azuresearch.html#AzureSearch.similarity_search

In [ ]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores.azuresearch import AzureSearch

# OpenAI API data (for embeddings)
openai_api_key = os.getenv("OPENAI_API_KEY")
openai_api_version = "2023-05-15"
model = "text-embedding-ada-002"

# Azure AI Search data (for vector store)
vector_store_address = os.getenv("AZURESEARCH_ENDPOINT") 
vector_store_password = os.getenv("AZURESEARCH_ADMIN_KEY")
vector_store_index = os.getenv("AZURESEARCH_INDEX_NAME")

# Initialize the embeddings
embeddings = OpenAIEmbeddings(
    openai_api_key=openai_api_key, 
    openai_api_version=openai_api_version, 
    model=model
)

# Initialize the vector store
vector_store = AzureSearch(
    azure_search_endpoint=vector_store_address,
    azure_search_key=vector_store_password,
    index_name=vector_store_index,
    embedding_function=embeddings.embed_query,
)

retriever = vector_store.as_retriever(k=4)
# AISearch retriver https://python.langchain.com/v0.2/docs/integrations/retrievers/azure_ai_search/

In [6]:
# query = "What are the fully local agents?"
# query = "What is Nvidia NIM API?"
query = "What is a generator function?"
docs = retriever.invoke(query)
for doc in docs:
    print(doc.page_content)

What is collections module and give examples.
This module implements specialized container datatypes providing alternatives 
to Python’s general purpose built-in containers, dict, list, set, and tuple.
Ex.1: Counter – dict subclass for counting hashable objects
Ex.2: defaultdict – dict subclass that calls a factory function to supply missing values
Ex.3: namedtuple – factory function for creating tuple subclasses with named fields
####

What are generator expression and why use them?
gen = (costly_fn(data) for data in iterable)
next(gen)
They behave a bit like a "lazy list comprehension."
Use them if you want data-on-demand, but you might not want to compute all of it at once in memory
As a contract needle in [expensive_fn(item) for item in haystack]
will evaluate the entire list comprehension, and only then begins the search
Note generator expressions use parentheses instead of square brackets
####

What would be the output of this code?
ex = (x ** 2 for x in range(10))
16 in ex
>>> ?

In [ ]:
#from https://github.com/langchain-ai/langchain/discussions/18752

prompt = "Why is the sky blue?"

# Use a "regular" similarity search, but with the hybrid option
docs = vector_store.similarity_search(
    query=prompt,
    k=10,
    search_type="hybrid",
    filters=f"source eq 'A'" #If you want to use filters
)

# Create the RAG chain
rag_chain_from_docs = (
    RunnablePassthrough.assign(context=(lambda x: format_docs(x["context"])))
    | prompt_template
    | llm
    | StrOutputParser()
)

# Prepare the input for the RAG chain
rag_input = {"query": prompt, "context": docs}

# Generate the answer using the RAG chain
rag_output = rag_chain_from_docs.invoke(rag_input)

# Create the final output dictionary manually
answer = {
    "context": docs,
    "question": prompt,
    "answer": rag_output
}

In [10]:
# Simpler solution:
# https://stackoverflow.com/questions/78576496/hybrid-search-using-azure-ai-search-and-lang-chain-as-a-retriever